# Double precision feather data are not identical in pandas and R


* Generate fake data

* Compare column means

* Compare column sums of squares to magnify and accumulate differences


In [1]:
%%bash
echo "R: $(which R)"
echo "python3: $(which python3)"
echo "jupyter: $(which jupyter)"

R: /home/turbach/.conda/envs/local_R/bin/R
python3: /mnt/cube/home/turbach/TPU_Projects/mixed/.venv/bin/python3
jupyter: /mnt/cube/home/turbach/TPU_Projects/mixed/.venv/bin/jupyter


In [2]:
%load_ext rpy2.ipython

/mnt/cube/home/turbach/TPU_Projects/mixed/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/mnt/cube/home/turbach/TPU_Projects/mixed/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
%%R
library(dplyr)
library(feather)
library(tibble)

show(R.version)
show(packageVersion('dplyr'))
.libPaths()

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




               _                           
platform       x86_64-conda_cos6-linux-gnu 
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          3                           
minor          5.3                         
year           2019                        
month          03                          
day            11                          
svn rev        76217                       
language       R                           
version.string R version 3.5.3 (2019-03-11)
nickname       Great Truth                 
[1] ‘0.8.5’
[1] "/mnt/cube/home/turbach/.conda/envs/local_R/lib/R/library"


In [4]:
import numpy as np
import pandas as pd
display(np.__version__)
display(pd.__version__)

'1.18.1'

'1.0.2'

In [5]:
data = pd.read_feather("data.feather")

mn = [ f"{type(ss)}: {ss:.32f}" for ss in (data[['x', 'y', 'z']]).mean()]
SS = [ f"{type(ss)}: {ss:.12f}" for ss in (data[['x', 'y', 'z']]**2).sum()]
for d in [data.shape, data.head(), data.tail(), mn, SS]:
    display(d)

(4096, 5)

,a,b,x,y,z
0,a1,b1,1.764052,1.629475,-0.528919
1,a1,b2,0.400157,-0.099045,-1.978994
2,a2,b1,0.978738,-0.201992,-0.093651
3,a2,b2,2.240893,-1.448826,-0.553708
4,a1,b1,1.867558,-1.714165,0.157762


,a,b,x,y,z
4091,a2,b2,-0.743152,0.094388,-0.211424
4092,a1,b1,0.891968,-0.666796,1.173267
4093,a1,b2,0.463873,0.666045,0.509159
4094,a2,b1,0.617661,0.532039,-0.158269
4095,a2,b2,2.496417,-1.050808,0.691676


["<class 'float'>: -0.02662872001551538220232018261413",
 "<class 'float'>: -0.00475772800525307761748683432756",
 "<class 'float'>: -0.01697831854605093282462036086145"]

["<class 'float'>: 3962.493575947415",
 "<class 'float'>: 4025.389704460528",
 "<class 'float'>: 4114.288326918832"]

In [6]:
%%R
options(pillar.sigfig=10)
data_R = read_feather("data.feather")
SS = data_R %>% summarize(SS_x=sum(x^2), SS_y=sum(y^2), SS_z=sum(z^2))
mn = data_R %>% summarize(mn_x=mean(x), mn_y=mean(y), mn_z=mean(z))
for (d in list(dim(data_R), head(data_R), tail(data_R))){
    show(d)
}
for (d in list(SS, mn)){
    for (v in d){
        print(sprintf("%.32f ", v))
    }
}

[1] 4096    5
# A tibble: 6 x 5
  a     b                 x               y               z
  <chr> <chr>         <dbl>           <dbl>           <dbl>
1 a1    b1     1.764052346    1.629475223    -0.5289185446 
2 a1    b2     0.4001572084  -0.09904470169  -1.978993792  
3 a2    b1     0.9787379841  -0.2019919553   -0.09365060026
4 a2    b2     2.240893199   -1.448825889    -0.5537081572 
5 a1    b1     1.867557990   -1.714164687     0.1577619546 
6 a1    b2    -0.9772778799  -0.04964157078   0.3714364503 
# A tibble: 6 x 5
  a     b                 x               y             z
  <chr> <chr>         <dbl>           <dbl>         <dbl>
1 a2    b1    -0.4383561662   1.581691225   -0.4006002018
2 a2    b2    -0.7431524812   0.09438784207 -0.2114236850
3 a1    b1     0.8919675820  -0.6667955311   1.173266879 
4 a1    b2     0.4638727652   0.6660448527   0.5091586300
5 a2    b1     0.6176608467   0.5320387664  -0.1582694868
6 a2    b2     2.496417063   -1.050807799    0.6916762561
[1] "3